1. Format lại địa chỉ  
Đầu vào sẽ là địa chỉ (lat,long) → đầu ra số nhà, đường, phường, quận, thành phố.


In [ ]:
import requests

def get_location(lat, long):
    api_keys = ["mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M",
                "br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
                "spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
                "0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
                "UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
                "c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE"]
    current_key_index = 0
    api_key = api_keys[current_key_index]
    url = f'https://revgeocode.search.hereapi.com/v1/revgeocode?apikey={api_key}&at={lat},{long}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách
        key_count = 0
        while response.status_code == 429:
            key_count += 1
            if key_count == len(api_keys):
                return { "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
            current_key_index = current_key_index + 1
            api_key = api_keys[current_key_index]
            url = f'https://revgeocode.search.hereapi.com/v1/revgeocode?apikey={api_key}&at={lat},{long}'
            response = requests.get(url)
            
        # Xử lý kết quả và trả về
        data = response.json()
        address = data['items'][0]['address']
        city = address.get('county', '')
        district = address.get('city', '')
        ward = address.get('district', '')
        street = address.get('street', '')
        houseNumber = address.get('houseNumber', '')
        result = { "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
        result = { "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
    
    return result


In [ ]:
# Đọc file đầu vào
df1 = pd.read_csv("")
# Lặp qua tất cả các cặp giá trị (lat, long)
address_info_list = []
for lat, long in zip(df1["lat"], df1["lng"]):
    address_info = get_location(lat, long)
    address_info_list.append(address_info)

# Tạo ra một DataFrame mới từ list này và thêm vào DataFrame gốc
address_df = pd.DataFrame(address_info_list)
df5 = pd.concat([df1, address_df], axis=1)

2. Cleaning địa chỉ gian hàng KV
Đầu vào sẽ là địa chỉ free text → đầu ra số nhà, đường, phường, quận, thành phố.

In [ ]:
# Đọc file đầu vào
df2 = pd.read_csv("")
# Lặp qua tất cả address
address_info_list = []
for address in df2["address"]:
    address_info = get_location(address)
    address_info_list.append(address_info)

# Tạo ra một DataFrame mới từ list này và thêm vào DataFrame gốc
address_df = pd.DataFrame(address_info_list)
df10 = pd.concat([df2, address_df], axis=1)

In [1]:
def cleaning_street(street):
    if isinstance(street, str):
        st = re.sub("(Hẻm|Ngõ|Ngách) \d+\/*\d*\/*\d* ", "", street)
        st = re.sub("(Hẻm T\d+)|Hẻm \d+[A-Za-z] ", "", st)
        st = re.sub(r"(30\/4|30\s(th[aá]ng\s)?4|30\s(th[aá]ng\s)?tư)", "Đường 30 Tháng 4", st, flags=re.IGNORECASE)
        st = re.sub(r"(3\/2|3\s(th[aá]ng\s)?2|3\s(th[aá]ng\s)?hai)", "Đường 3 Tháng 2", st, flags=re.IGNORECASE)
        st = re.sub(r"\bQL\.", "Quốc Lộ ", st)
        st = re.sub("-", "/", st)
        st = st.strip()  # Loại bỏ khoảng trắng ở hai đầu của đường dẫn
        lower_text = st.lower()
        if not re.search(r'^chung cư|^hẻm|^ấp|^khu phố|^phố|^quốc lộ|^tổ', lower_text) and not re.search(r'^đường', lower_text):
            st = "Đường " + st
        return st
    return street

In [ ]:
df5["cleaned_street"] = df5.apply(lambda x: cleaning_street(x.Street), axis=1)

3. Cleaning lại tên đường và ngành hàng

In [ ]:
def format_industry(row):
    name = row['name']
    industry = row['industry']
    if industry in ["Quán ăn", "Nhà hàng"] and re.search(r'cơm|lẩu|nướng|quán|cháo|sườn|phở|bún|miến|nhậu|bia', name.lower()):
        return "Quán ăn"
    elif re.search(r'buffet', name.lower()):
        return "Buffet"
    elif industry == "Trà sữa/Cà phê/Đồ uống" and re.search(r'coffee|cà phê|cafe|cofe|coffe|cofee|ca phe|café', name.lower()):
        return "Cà phê"
    elif industry == "Trà sữa/Cà phê/Đồ uống":
        return "Trà sữa & Đồ uống"
    else:
        return industry

In [ ]:
df5['industry_fix'] = df5.apply(format_industry, axis=1)


4. Mapping gian hàng Kv và gian hàng Google


In [ ]:
# df1: Tập gian hàng google
#Tạo key_1 và key_2 
df1['key_1'] = df1.apply(lambda row: '|'.join([str(row[col]) for col in ['House_number', 'cleaned_street', 'Ward', 'District', 'City']]), axis=1)
df1['key_2'] = df1.apply(lambda row: '|'.join([str(row[col]) for col in ['cleaned_street', 'Ward', 'District', 'City']]), axis=1)




In [ ]:
# df1: Tập gian hàng Kv
df2['key_1'] = df2.apply(lambda row: '|'.join([str(row[col]) for col in ['House_number', 'cleaned_street', 'Ward', 'District', 'City']]), axis=1)
df2['key_2'] = df2.apply(lambda row: '|'.join([str(row[col]) for col in ['cleaned_street', 'Ward', 'District', 'City']]), axis=1)

In [ ]:
# Tạo một từ điển để lưu trữ giá trị potentialid cho mỗi giá trị key_1 và key_2 trong DataFrame df2
potentialid_dict = {}

# Lặp lại qua DataFrame df2 và điền các giá trị potentialid vào từ điển
for i, row in df2.iterrows():
    if row['key_1'] not in potentialid_dict:
        potentialid_dict[row['key_1']] = []
    potentialid_dict[row['key_1']].append(row['potentialid'])
    
    if row['key_2'] not in potentialid_dict:
        potentialid_dict[row['key_2']] = []
    potentialid_dict[row['key_2']].append(row['potentialid'])
# Định nghĩa biến used_potentialid
used_potentialid = []
# Lặp lại qua mỗi hàng trong DataFrame df1
for i, row in df1.iterrows():
    # Kiểm tra xem giá trị key_1 có tồn tại trong df2 hay không
    if row['key_1'] in potentialid_dict:
        # Điền các giá trị potentialid từ từ điển vào
        potentialid_list = potentialid_dict[row['key_1']]
    # Nếu giá trị key_1 không tồn tại trong df2, kiểm tra xem giá trị key_2 có tồn tại hay không
    elif row['key_2'] in potentialid_dict:
        # Điền các giá trị potentialid từ từ điển vào
        potentialid_list = potentialid_dict[row['key_2']]
    else:
        # Nếu cả key_1 và key_2 đều không tồn tại trong df2, bỏ qua hàng này
        continue
        
    # Kiểm tra xem có bất kỳ giá trị potentialid nào đã được sử dụng chưa
    unused_potentialid_list = [pid for pid in potentialid_list if pid not in used_potentialid]
    if not unused_potentialid_list:
        continue
    
    # Nếu có giá trị potentialid chưa được sử dụng, điền giá trị đầu tiên vào df1 và thêm vào danh sách đã sử dụng
    potentialid = unused_potentialid_list[0]
    df1.loc[i, 'potentialid'] = potentialid
    used_potentialid.append(potentialid)

Bảng data gian hàng google sau khi xử lý: kiotvietbi-prelive.kv_datawarehouse.here_fnb_cantho_final

Bảng data gian hàng KV sau khi xử lý: kiotvietbi-prelive.kv_datawarehouse.kv_fnb_cantho_2_final



Clean địa chỉ API sử dụng free text:

In [1]:
list_api_keys = ["1EotnhALO_Xf7d5mlOVefECdfpbgPG_Dik0TeOcQauQ",
"92hxCAU523jWUFRXMhM-p3w66P3PxYfY_tZm9pPeShU",
"KwvOafjkobVhF8Iz63lMyNooS-hjaYgmHdZrqVVVHWw",
"N5qb2q5dSfly3fnqWMsDPqVMWUd1yRbTBxbT34hvw_g",
"SsQNhtNfemZJr_FzCYZlub29VeTTjymT3Qm6syC50W8",
"br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
"spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
"0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
"UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
"c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE",
"WCpDyLiAACoL9zwSFQQa_HboP-HdWXSM601W1N02aeg",
"faJrm5egL12NmLivrDsbqz44I47DhbJQUIg89L3TdIo",
"cdXQHZ3zzsZv-n04kjimcml7e3BRKcR2H3wl8XPi6SU",
"RBRLSkMgpwy6QF0jKsXs-jXEliqtAwwD_jd0DbHrEN8",
"Dd-n66bTKOIKeQ0H6nRvIOMWatnJ_viPxKMNGpxsFa4",
"q-Ao_RRfZNyfQPp2XsnYGbrOItF6SlXN0jObPIKVnk",
"ufLjrWIrvjyLHzL1TZIEOZjKpgolDMBUPXUwoIblgs0",
"tvjh1g0Xygp6DMsH8n5hX41jgbNh3VE61MsnWHYkfZw",
"d_8_FhktHjeMh91gB9bBKGHgMJFC6auZqHThhlHL9Hw",
"P1RAwTjkXvZUQLjHyi06Zm1YP1KeaZMJdjaNN1ZeOeE",
"DQV3KQ2U9DLoro0jMQUoIr-PQMKBpKIEpOqXzF1A498",
"6IcCytBh4PkKOC3YE2Smw4IGdAyTD7XTtsmgtxF-kdY",
"PgxoArILmRDFQreVp3pC6bexYfcKIVwMMN4oKpa8VOo",
"mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"]

In [2]:
# Hàm gọi api here để lấy thông tin vị trí từ địa chỉ chi tiết
import requests

def get_location(address, api_key ):
    #api_key = "mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M"

    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách

        if response.status_code == 429:
            return {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
        else: 
            data = response.json()    
            # Xử lý kết quả và trả về
            if data == {'items': []}: 
                list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "True" }
            else: 
                result = data['items'][0]
                position = result['position']
                lat = position.get('lat', '')
                long = position.get('lng', '')
                address_info = result['address']
                city = address_info.get('county', '')
                district = address_info.get('city', '')
                ward = address_info.get('district', '')
                street = address_info.get('street', '')
                houseNumber = address_info.get('houseNumber', '')
                list_address = {"lat": lat, "lng": long, "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
           # Nếu thử xảy ra lỗi trả ra tập rỗng và progess là False đánh dấu những case chưa xử lí
        list_address = {"lat": "", "lng": "", "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
           
    return list_address

# Hàm xử lí dữ liệu đầu vào là dataframe
def out_put(df, api_key):
    house_numbers = []
    streets = []
    wards = []
    districts = []
    cities = []
    progresses = []
    
    for address in df["address"]:
        address_info = get_location(address,api_key)
        house_numbers.append(address_info["House_number"])
        streets.append(address_info["Street"])
        wards.append(address_info["Ward"])
        districts.append(address_info["District"])
        cities.append(address_info["City"])
        progresses.append(address_info["progress"])
    
    df["House_number"] = house_numbers
    df["Street"] = streets
    df["Ward"] = wards
    df["District"] = districts
    df["City"] = cities
    df["progress"] = progresses

    return df

In [3]:
df_test = df


NameError: name 'df' is not defined

In [ ]:
# Calculate the number of groups
num_groups = len(df_test) // 1000

# Calculate the number of remaining rows
remaining_rows = len(df_test) % 1000

# Create a list to store the new dataframes
dfs = []

# Split the dataframe into groups of two rows each
for i in range(num_groups):
    start_index = i * 1000
    end_index = start_index + 1000
    df_group = df_test[start_index:end_index].copy()
    dfs.append(df_group)

# Add the remaining rows to the last dataframe
if remaining_rows > 0:
    last_df = df_test[-remaining_rows:].copy()
    dfs.append(last_df)

In [ ]:
# tiến hành chạy multi thread
import threading
import time

final = []
def out_put_with_time(df, api_key):
    start_time = time.time()
    out_put(df, api_key)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Time: {execution_time/60} phút")

for df , api_key in zip(dfs, list_api_keys):
    thread = threading.Thread(target=out_put_with_time, args=(df,api_key))
    thread.start()
    thread.join()
    final.append(df)

print("Done")

In [ ]:
merged_df = pd.concat(final, ignore_index=True)
